In [14]:
pip install loguru

Note: you may need to restart the kernel to use updated packages.


In [20]:
import torch
from PIL import Image

from torch import nn, save, load
from torch.optim import Adam    
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from loguru import logger


train = datasets.MNIST(
    root='data',
    download=True,
    train=True,
    transform=ToTensor()
)
datasets = DataLoader(train, 32)

class ImageClassifier(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, (3,3)),
            nn.ReLU(),
            nn.Conv2d(32, 64,(3,3)),
            nn.ReLU(),
            nn.Conv2d(64, 64,(3,3)),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64*(28-6) * (28 - 6), 10)
        )

    def forward(self, x):
        return self.model(x)
    
clf = ImageClassifier().to('mps')
opt = Adam(clf.parameters(), lr=1e-3)

loss_function = nn.CrossEntropyLoss()

if __name__ == '__main__':
    with open('model_state.pt', 'rb') as f:
        clf.load_state_dict(load(f))
    img = ToTensor()(Image.open('img_3.jpg')).unsqueeze(0).to('mps')

    logger.info(torch.argmax(clf(img)))
    # for epoch in range(10):
    #     for batch in datasets:
    #         X,y = batch
    #         X,y = X.to('mps'), y.to('mps')
    #         yhat = clf(X)
    #         loss = loss_function(yhat, y)

    #         opt.zero_grad()
    #         loss.backward()
    #         opt.step()
    #     logger.info(f'Epoch:{epoch} loss is {loss.item()}')

    # with open('model_state.pt', 'wb') as f:
    #     save(clf.state_dict(), f)
        



2025-04-07 19:54:19.220 | INFO     | __main__:<module>:47 - tensor(9, device='mps:0')


In [9]:
import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [10]:
import platform
import torch

# Check CPU architecture
arch = platform.machine()
print(f"CPU architecture: {arch}")
if "arm" in arch.lower() or "aarch64" in arch.lower():
    print("✅ ARM-based Mac detected (Apple Silicon)")
else:
    print("⚠️ Not an ARM-based Mac")

# Check for MPS (Apple Metal GPU backend)
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✅ MPS (Metal GPU backend) is available and will be used")
else:
    device = torch.device("cpu")
    print("⚠️ MPS not available, falling back to CPU")

# Simple test tensor operation
x = torch.randn(1000, 1000, device=device)
y = torch.randn(1000, 1000, device=device)
z = x @ y  # matrix multiplication

print(f"Tensor operation successful on: {device}")
print(f"Result tensor shape: {z.shape}")

CPU architecture: arm64
✅ ARM-based Mac detected (Apple Silicon)
✅ MPS (Metal GPU backend) is available and will be used
Tensor operation successful on: mps
Result tensor shape: torch.Size([1000, 1000])
